In [2]:
import pandas as pd
from pathlib import Path
import json
data_dir = Path('semantic_search/data/abo/')
csv = data_dir / 'images' /'metadata'/'images.csv.gz'
listings_dir = data_dir/ 'listings'/'metadata'

In [27]:
import redis
from redis.commands.json.path import Path as redPath

r = redis.Redis(host="localhost", port=6379, charset="utf-8", decode_responses=True)

In [11]:
df = pd.read_csv(csv)
df.head()

,image_id,height,width,path
0,010-mllS7JL,106,106,14/14fe8812.jpg
1,01dkn0Gyx0L,122,122,da/daab0cad.jpg
2,01sUPg0387L,111,111,d2/d2daaae9.jpg
3,1168jc-5r1L,186,186,3a/3a4e88e6.jpg
4,11RUV5Fs65L,30,500,d9/d91ab9cf.jpg


In [7]:
listing_jsons = [f for f in listings_dir.iterdir()]

In [8]:
from pandas import isna


table_name = "listings"

redis_file_name = "redis_populate_listings.txt"

table_exists = False
def extract_values(x):
    if isinstance(x, list) and 'value' in x[0]:
            return [a['value'] for a in x]    
    if pd.isna(x):
        return ''
    else:
        return x


list_detail = {}
descriptors = ['item_id', 'item_name', 'model_name', 'brand', 'bullet_point']
with open(redis_file_name, 'w') as redis_cmd:
    for listing_file in listing_jsons[:1]:
        df_list = pd.read_json(listing_file, lines=True)
        pipe = r.pipeline()
        for _, row in df_list.iterrows():
            feature = json.dumps({d: extract_values(row[d]) for d in descriptors})
            if not pd.isna(row['main_image_id']):
                key = "IMG:" + row['main_image_id']
                # pipe.json().set(key, redPath.root_path(), feature)
                # redis_cmd.write(f"JSON.SET {key} . {json.dumps(feature)}\n")
                
            try:
                for img in row['other_image_id']:
                    key = "IMG:" + img
                    # pipe.json().set(key, redPath.root_path(), feature)
                    # redis_cmd.write(f"JSON.SET {key} . {json.dumps(feature)}\n")

            except TypeError:
                # ignore nans
                pass
        print("Starting to update redis")
        # pipe.execute()




Starting to update redis


In [6]:
# import redis
# from redis.commands.json.path import redPath

# r = redis.Redis(host="redis-19391.c15.us-east-1-2.ec2.cloud.redislabs.com", port=19391, password="J7bPRPDU7CrgGDxzEUZhZym7c48Ksp4Z")
# r.json().set("mykey", ".", {"hello": "world", "i am": ["a", "json", "object!"]})
# jane = {
#      'name': "Jane",
#      'Age': 33,
#      'Location': "Chawton"
#    }

# r.json().set('person:1', Path.root_path(), jane)

# result = r.json().get('person:1')
# print(result)

In [12]:
df_list['main_image_id'][0]

'718mYsQTQbL'

In [8]:
# import json
# import gzip  

# with gzip.open(listing_jsons[0], 'r') as fin:        # 4. gzip
#     json_bytes = fin.read() 
#     print(json_bytes)
#     print  
#     for jbyte in json_bytes:
#         print(jbyte)
#         break                   # 3. bytes (i.e. UTF-8)
#         json_str = jbyte.decode('utf-8')            # 2. string (i.e. JSON)
#         data = json.loads(json_str)    

In [4]:
r.json().get('IMG:0af115ee')

In [5]:
df_list['main_image_id'][0]

NameError: name 'df_list' is not defined

In [13]:
df[df['path'] == '0a/0af115ee.jpg']

,image_id,height,width,path
235222,71vZjI121nL,2000,2000,0a/0af115ee.jpg


In [18]:
df['path'][:10].str.extract(r'\/(.*).jpg')

,0
0,14fe8812
1,daab0cad
2,d2daaae9
3,3a4e88e6
4,d91ab9cf
5,20098c4d
6,9987a1c8
7,89a2ff4d
8,ee239f0f
9,75536bf9


In [19]:
df['name'] = df['path'].str.extract(r'\/(.*).jpg')

In [25]:
pipe = r.pipeline()
for _, (name, id) in df[['name', 'image_id']].iterrows():
    pipe.set('MAP:'+ str(name), id )
pipe.execute()

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [23]:
next(df[['name', 'image_id']].iterrows())

(0,
 name           14fe8812
 image_id    010-mllS7JL
 Name: 0, dtype: object)

In [28]:
r.get('MAP:14fe8812')

'010-mllS7JL'

In [32]:
r.json().get('IMG:010-mllS7JL')

'{"item_id": "B07Y7M8LV7", "item_name": ["Glasses Old Fashioned, 6-Piece, 350ml \\u2026"], "model_name": "", "brand": ["UMI"], "bullet_point": ""}'